## Screen Time Post Processing

The purpose of the notebook is to process the screen timing report, which is used demarcate the start/stop time for each eye tracked screen in this experiment.

In [31]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series, ExcelFile, ExcelWriter
import datetime
import time

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## Number of Eye Tracked Screens

In [32]:
tracked_screens_phase_1 = 9
tracked_screens_phase_2 = 2
tracked_screens_phase_3 = 8

total_timed_screens = tracked_screens_phase_1 + tracked_screens_phase_2 + tracked_screens_phase_3
num_screens = 2*total_timed_screens

## Recorded Screen Start/Stop Time

The screen timing report file consists initially of 4 columns, the Subject ID, Timer Action (Start/Stop), and the Timestamp (Python). The screen labels must be appended manually.

In [33]:
screen_start_stop_timing = pd.read_csv('session_data/timing_subject_1.csv')
screen_start_stop_timing

,subject,action,screen,timestamp
0,1,start,1,1.513359e+09
1,1,stop,1,1.513359e+09
2,1,start,1,1.513359e+09
3,1,stop,1,1.513359e+09
4,1,start,1,1.513359e+09
5,1,stop,1,1.513359e+09
6,1,start,1,1.513359e+09
7,1,stop,1,1.513359e+09
8,1,start,1,1.513359e+09
9,1,stop,1,1.513359e+09


### Test: Confirm that the number of start/stop events per screen is correct

In [34]:
# Test lenth of click report file
assert len(screen_start_stop_timing) == num_screens

### Pair Ordering for Subject 1 in Phase 2 and 3

In [35]:
order = [3,4,9,6,8,7,2,5,1]

### Phase 1 Pair Order

In [36]:
p1 = pd.Series(np.repeat(np.arange(1,10), 2))

### Phase 2 Pair Order

In [37]:
p2 = pd.Series(np.repeat(order[0], 4))

### Phase 3 Pair Order

In [38]:
p3 = pd.Series(np.repeat(order[1:], 2))

### Merge Pairs for Phase 1 - 3

In [39]:
pair_order = p1.append(p2).append(p3).reset_index(drop=True)

### Merge the Game Pair Order and Screen Timing

In [40]:
screen_start_stop_timing['pair_id'] = pair_order
screen_start_stop_timing

,subject,action,screen,timestamp,pair_id
0,1,start,1,1.513359e+09,1
1,1,stop,1,1.513359e+09,1
2,1,start,1,1.513359e+09,2
3,1,stop,1,1.513359e+09,2
4,1,start,1,1.513359e+09,3
5,1,stop,1,1.513359e+09,3
6,1,start,1,1.513359e+09,4
7,1,stop,1,1.513359e+09,4
8,1,start,1,1.513359e+09,5
9,1,stop,1,1.513359e+09,5


### Load the Eye Tracking Data File

In [41]:
gaze = screen_start_stop_timing = pd.read_csv('session_data/subject_1_all_gaze.csv')
gaze

,MEDIA_ID,MEDIA_NAME,CNT,TIME(2017/12/15 12:26:37.409),TIMETICK(f=3328125),FPOGX,FPOGY,FPOGS,FPOGD,FPOGID,...,RPCX,RPCY,RPD,RPS,RPV,BKID,BKDUR,BKPMIN,AOI,Unnamed: 32
0,0,NewMedia0,0,0.00000,2773055138,0.21463,0.28561,0.00000,0.00000,1,...,0.76706,0.60058,15.80494,0.99245,1,0,0.0,12,NaN,NaN
1,0,NewMedia0,1,0.01642,2773109769,0.21458,0.28474,0.00000,0.01642,1,...,0.76705,0.60008,15.80904,0.98184,1,0,0.0,12,NaN,NaN
2,0,NewMedia0,2,0.03235,2773162829,0.21569,0.28609,0.00000,0.03235,1,...,0.76710,0.59952,15.44995,0.97123,1,0,0.0,12,NaN,NaN
3,0,NewMedia0,3,0.04883,2773217551,0.21602,0.28719,0.00000,0.04883,1,...,0.76737,0.59973,15.27540,0.96063,1,0,0.0,12,NaN,NaN
4,0,NewMedia0,4,0.06531,2773272481,0.21605,0.28745,0.00000,0.06531,1,...,0.76763,0.60002,15.75126,0.99752,1,0,0.0,12,NaN,NaN
5,0,NewMedia0,5,0.08215,2773328536,0.21539,0.28568,0.00000,0.08216,1,...,0.76789,0.60010,15.54790,1.04502,1,0,0.0,12,NaN,NaN
6,0,NewMedia0,6,0.09857,2773383173,0.21435,0.28200,0.00000,0.09857,1,...,0.76795,0.60055,15.92397,1.09252,1,0,0.0,12,NaN,NaN
7,0,NewMedia0,7,0.11499,2773437788,0.21304,0.27781,0.00000,0.11499,1,...,0.76797,0.60101,15.85170,1.10313,1,0,0.0,12,NaN,NaN
8,0,NewMedia0,8,0.13147,2773492678,0.20754,0.24902,0.00000,0.11499,1,...,0.76816,0.60050,15.47580,1.06623,1,0,0.0,12,NaN,NaN
9,0,NewMedia0,9,0.14783,2773547191,0.19976,0.24398,0.00000,0.11499,1,...,0.76822,0.60059,15.45307,1.02934,1,0,0.0,12,NaN,NaN


### Retrieve the Start Time

In [42]:
start_time = list(gaze)[3]
start_time

'TIME(2017/12/15 12:26:37.409)'

### Convert Start Time to Timestamp

In [43]:
date_time_text = start_time[5:-1]
date = datetime.datetime.strptime(date_time_text, "%Y/%m/%d %H:%M:%S.%f")

In [46]:
date.timestamp()

1513358797.409

### ToDo

* Filter out rows not corresponding to game screens